<a href="https://colab.research.google.com/github/aubricot/CV_for_flower_CT/blob/main/preprocessing_anth_ov_nrrd2nifti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert .nrrd and .seg.nrrd files from 3D Slicer to .nii.gz (nifti) files for machine learning - Anther and Ovary
---   
*Last Updated 22 March 2025*  
-Runs in Python 3-   
Convert 3D slicer volume and segmentation files into nifti format for machine learning

In [ ]:
#@title Choose where to save results

# Imports
import os
!pip install slicerio
import slicerio
import json
!pip install SimpleITK
import SimpleITK as sitk
import nrrd

# Use dropdown menu on right
save = "in my Google Drive" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]

# Mount google drive to export image tagging file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Type in the path to your project wd in form field on right
wd = "/content/drive/MyDrive/slicer/anth_ov_seg" # @param ["/content/drive/MyDrive/slicer"] {"allow-input":true}
print("\nWorking with data from:")
%cd $wd

# Set up folder to save .nii.gz files
ngz_folder = "anther_ov_nii_gzs" # @param {"type":"string","placeholder":"nii_gzs"}
ngz_fpath = wd + '/' + ngz_folder
# Set up folder to save segmentation files (labels)
labels_fpath = ngz_fpath + '/' + 'labels'

# Make ngz_folder if it doesn't already exist
if not os.path.exists(ngz_fpath):
    print("Making ngz_folder at: ", ngz_fpath)
    os.makedirs(ngz_fpath)
    print("Making labels_folder at : ", labels_fpath)
    os.makedirs(labels_fpath)
else:
    print("\nngz_folder and labels_folder already exist at: ", ngz_fpath, labels_fpath)

In [ ]:
#@title Anther/Ovary: Crawl through wd to find .nrrd and _crop.nrrd files, convert them to .nii.gz, save to ngz_folder

# List input and output filenames
in_vols = [] # volume files (.nrrd or _crop.nrrd)
out_vols = [] # volume files (.nii.gz)
in_segs = [] # segmentation files (.seg.nrrd)
out_segs = [] # segmentation files (_seg.nii.gz)

# Make a list of directories to check if missing files
vols_to_check = []
segs_to_check = []

seg_files = []
vol_files = []

# Walk through chosen directories
dir = wd
print("\n \033[92m Searching files in: {}\033[0m".format(dir))
files = os.listdir(dir)
for file in files:
    if file.endswith(".seg.nrrd"):
        seg_files.append(file)
    elif file.endswith(".nrrd"):
        vol_files.append(file)

# Segs
for seg_file in seg_files:
    try:
        print("\033[92m Found segmentation file!\033[0m")
        print("seg_file: ", seg_file)
        # First, check if it has the correct number of labels
        num_labels = 2 #@param
        seg = slicerio.read_segmentation(dir + '/' + seg_file, skip_voxels=True)
        num_segs = len(seg["segments"])
        if num_segs == num_labels:
                img = sitk.ReadImage(dir + '/' + seg_file)
                outpath = labels_fpath + '/' + seg_file[:-9] + '_seg.nii.gz'
                print("seg_file saved to: ", outpath)
                sitk.WriteImage(img, outpath)
                in_segs.append(seg_file)
                out_segs.append(outpath)
        else:
                print("\033[31m Wrong number of labels for segmentation file: {}\033[0m\n".format((dir + '/' + seg_file)))
                segs_to_check.append((dir + '/' + seg_file))
    except:
        pass

# Vols
for vol_file in vol_files:
    try:
        print("\033[92m Found volume file!\033[0m")
        print("vol_file: ", vol_file)
        img = sitk.ReadImage(dir + '/' + vol_file)
        outpath = ngz_fpath + '/' + vol_file[:-5] + '.nii.gz'
        print("vol_file saved to: ", outpath)
        sitk.WriteImage(img, outpath)
        in_vols.append(vol_file)
        out_vols.append(outpath)
    except:
        pass

    print("{} .nrrd files and {} seg.nrrd files from {} converted to .nii.gz at \n volumes: {} \n labels: {}".format(len(in_vols), len(in_segs), wd, ngz_fpath, labels_fpath))

In [ ]:
#@title Zip the folder for archival purposes
# Enter wd, zip and unzipped filename using form field to right
wd = "/content/drive/MyDrive/slicer/anth_ov_seg" #@param ["/content/drive/MyDrive"] {allow-input: true}
zip_fn = ngz_folder + '.zip'
zip_fpath = wd + '/' + zip_fn
unzipped_fn = ngz_folder
unzipped_fpath = wd + '/' + unzipped_fn

# Zip the file/folder
!zip -r -j $zip_fpath $unzipped_fpath